In [0]:
 # Install keras for neural network model
!pip install -q keras
import keras

# Install Alpha Vantage
! pip install alpha_vantage

# Install fixed yahoo finance
! pip install fix_yahoo_finance

# Install technical analysis library
! pip install pyti

# Install Quandl
! pip install Quandl
import quandl
quandl.ApiConfig.api_key = "13iHgug8oDVAv_tD8ax-"

# install pytrends
! pip install pytrends

# function for using plotly
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

# libraries
import fix_yahoo_finance as fyf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import sklearn.preprocessing
from alpha_vantage.timeseries import TimeSeries
ts=TimeSeries(key='0NQE5NPYNBWUATFT',output_format='pandas')
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from alpha_vantage.foreignexchange import ForeignExchange
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
# standard model
from keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU, Activation, LSTM
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau
from keras import regularizers
# using Plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True) 
# Google Trends
from pytrends.request import TrendReq


# technical analysis indicators
import pyti
from pyti.exponential_moving_average import exponential_moving_average as ema
from pyti.relative_strength_index import relative_strength_index
from pyti.accumulation_distribution import accumulation_distribution as accdis
from pyti.bollinger_bands import percent_b
from pyti.money_flow_index import money_flow_index
from pyti.momentum import momentum 
from pyti.stochastic import percent_d as stoc
from pyti.volatility import volatility
from pyti.on_balance_volume import on_balance_volume

Using TensorFlow backend.


    100% |████████████████████████████████| 61kB 3.3MB/s 
    100% |████████████████████████████████| 51kB 5.0MB/s 
    100% |████████████████████████████████| 2.2MB 568kB/s 
    100% |████████████████████████████████| 430kB 2.8MB/s 


    100% |████████████████████████████████| 102kB 8.1MB/s 
    100% |████████████████████████████████| 256kB 4.2MB/s 
  Running setup.py bdist_wheel for inflection ... - done
  Stored in directory: /content/.cache/pip/wheels/41/fa/e9/2995f4ab121e9f30f342fa2d43f0b27f851a0cb9f0d98d3b45
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /content/.cache/pip/wheels/95/14/9a/5e7b9024459d2a6600aaa64e0ba485325aff7a9ac7489db1b6
Successfully built inflection pycparser
    100% |████████████████████████████████| 5.7MB 236kB/s 


## VIX index data  


In [0]:
prices=quandl.get('CBOE/VXST')

# add Google Trends data regarding market sentiment
pytrends = TrendReq(hl='en-US', tz=360)
# function to get Google trend over time
def get_gtrend(keyword):
  pytrends.build_payload(kw_list=[keyword], cat=0, timeframe='all', geo='', gprop='')
  interest_over_time=pytrends.interest_over_time()[keyword]
  return(interest_over_time)
volatility_gtrend=get_gtrend('volatility')
volatility_gtrend=volatility_gtrend.asfreq('d').fillna(method='ffill')   # change frequency to daily and fill NAs with last available value

# merge google trends with stock prices
prices['volatility_gtrend']=volatility_gtrend

print(prices.head())

# plot VIX and gtrends
configure_plotly_browser_state()
vix=Scatter(x=prices.index,y=prices['Close'],name='VIX')
gtrend=Scatter(x=prices.index,y=prices['volatility_gtrend'],name='gtrend')
iplot([vix,gtrend])

            Open  High  Low  Close  volatility_gtrend
Date                                                 
2011-01-03   NaN   NaN  NaN  16.04               41.0
2011-01-04   NaN   NaN  NaN  16.06               41.0
2011-01-05   NaN   NaN  NaN  15.57               41.0
2011-01-06   NaN   NaN  NaN  15.71               41.0
2011-01-07   NaN   NaN  NaN  15.01               41.0


In [0]:
# select variables to use in the model
variables=['Close','volatility_gtrend']
data=prices[variables] 
data=data.sort_index(ascending=True)

# calculate daily return
data['DailyReturn']=data['Close'].pct_change()

# drop NA values and sort in descending order
data=data.dropna()
data=data.sort_index(ascending=False)

print('\ndata:\n',data.head())

# data[variables].plot(subplots=True)


data:
             Close  volatility_gtrend  DailyReturn
Date                                             
2018-03-01  23.89               51.0     0.244919
2018-02-28  19.19               68.0     0.059051
2018-02-27  18.12               68.0     0.195251
2018-02-26  15.16               68.0     0.012016
2018-02-23  14.98               68.0    -0.173745


In [0]:
# select input features and transform into numpy array
data_input_features=data[['DailyReturn','volatility_gtrend']]
input_data=np.asarray(data_input_features)
print('\ninput_data array:\n',input_data)

# select target and transform into numpy array
target=np.asarray(data['Close'])
print('\ntarget:',target)


input_data array:
 [[ 2.44919229e-01  5.10000000e+01]
 [ 5.90507726e-02  6.80000000e+01]
 [ 1.95250660e-01  6.80000000e+01]
 ...
 [ 8.99165061e-03  4.10000000e+01]
 [-3.05105853e-02  4.10000000e+01]
 [ 1.24688279e-03  4.10000000e+01]]

target: [23.89 19.19 18.12 ... 15.71 15.57 16.06]


In [0]:
# reshape input data into 3D format with lookback window
window=30       # lookback window
nb_samples=len(input_data)-window   # number of samples (windows) that we can extract from the data
# function to transform into 3D format
def input_features(nb_samples,LOOKBACK_WINDOW):
  input_list = [np.expand_dims(input_data[1+i:1+LOOKBACK_WINDOW+i,:], axis=0) for i in range(nb_samples)]   # lagged daily returns from T-1 to T-window+1+
  input_mat = np.concatenate(input_list, axis=0)
  return(input_mat)

input_mat=input_features(nb_samples,window)
print('\ninput_mat shape:',input_mat.shape)
print('\ninput_mat first sample:\n',input_mat[0])                                  # show first of the samples



input_mat shape: (1771, 30, 2)

input_mat first sample:
 [[ 5.90507726e-02  6.80000000e+01]
 [ 1.95250660e-01  6.80000000e+01]
 [ 1.20160214e-02  6.80000000e+01]
 [-1.73745174e-01  6.80000000e+01]
 [-9.39530235e-02  6.80000000e+01]
 [-3.05232558e-02  6.80000000e+01]
 [ 1.04925054e-01  6.80000000e+01]
 [ 5.38213132e-03  6.80000000e+01]
 [-8.00854245e-03  6.80000000e+01]
 [-2.59391064e-01  6.80000000e+01]
 [-5.73984346e-02  6.80000000e+01]
 [-1.52022756e-01  6.80000000e+01]
 [-2.36486486e-01  6.80000000e+01]
 [ 2.61107730e-01  6.80000000e+01]
 [-1.67257983e-01  6.80000000e+01]
 [-3.35018537e-01  6.80000000e+01]
 [ 2.14634146e+00  6.80000000e+01]
 [ 3.60750361e-01  6.80000000e+01]
 [ 0.00000000e+00  6.80000000e+01]
 [-1.44444444e-01  4.90000000e+01]
 [ 8.65191147e-02  4.90000000e+01]
 [ 4.25430210e-01  4.90000000e+01]
 [-1.01374570e-01  4.90000000e+01]
 [ 1.65938865e-02  4.90000000e+01]
 [ 3.90199637e-02  4.90000000e+01]
 [ 1.47329650e-02  4.90000000e+01]
 [ 4.32276657e-02  4.90000000e+0

In [0]:
# split data into train / test (remember that dates are inverted)
split=0.7
train_size=int(len(input_mat)*split)
test_size=len(input_mat)-train_size
trainX=input_mat[test_size:len(input_mat)]
testX=input_mat[0:test_size]

# create target variable as daily positive or negative return
trainY=target[test_size:len(input_mat)]
testY=target[0:test_size]
print('\ntrainX:',trainX.shape,'trainY:',trainY.shape)
print('target:',target)
print('testX:',testX.shape,'testY:',testY.shape,'\n')


trainX: (1239, 30, 2) trainY: (1239,)
target: [23.89 19.19 18.12 ... 15.71 15.57 16.06]
testX: (532, 30, 2) testY: (532,) 



In [0]:
# LSTM model, it requires 3D format of the input layer (samples, timesteps, features)
# help --> http://aaaquants.com/2017/08/27/machine-learning-lstm-networks/
bsize=128    #batch size
nfeatures=input_mat.shape[2]

# model building 
model = Sequential()
model.add(LSTM(30, input_shape=(window,nfeatures)))
model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='rmsprop')
model.fit(trainX,trainY,epochs=500,batch_size=bsize,validation_split=0.3)

# accuracy
score=model.evaluate(testX,testY,batch_size=bsize)
print('\nPerformance:',score)

Train on 867 samples, validate on 372 samples
Epoch 1/500
867/867 [==============================] - 1s 988us/step - loss: 223.1569 - val_loss: 537.2820
Epoch 2/500
867/867 [==============================] - 0s 401us/step - loss: 211.4919 - val_loss: 523.7216
Epoch 3/500
867/867 [==============================] - 0s 394us/step - loss: 204.3807 - val_loss: 513.3115
Epoch 4/500
867/867 [==============================] - 0s 379us/step - loss: 196.9048 - val_loss: 504.4693
Epoch 5/500
867/867 [==============================] - 0s 406us/step - loss: 191.4503 - val_loss: 496.1250
Epoch 6/500
867/867 [==============================] - 0s 378us/step - loss: 187.0397 - val_loss: 487.4083
Epoch 7/500
867/867 [==============================] - 0s 389us/step - loss: 180.3194 - val_loss: 475.8680
Epoch 8/500
867/867 [==============================] - 0s 373us/step - loss: 170.8331 - val_loss: 463.6559
Epoch 9/500
867/867 [==============================] - 0s 385us/step - loss: 166.1490 - val_loss: 

Epoch 20/500
867/867 [==============================] - 0s 397us/step - loss: 136.4975 - val_loss: 408.3420
Epoch 21/500
867/867 [==============================] - 0s 378us/step - loss: 134.1513 - val_loss: 405.2094
Epoch 22/500
867/867 [==============================] - 0s 414us/step - loss: 134.1338 - val_loss: 402.1098
Epoch 23/500
867/867 [==============================] - 0s 376us/step - loss: 131.4693 - val_loss: 399.0224
Epoch 24/500
867/867 [==============================] - 0s 387us/step - loss: 129.8973 - val_loss: 395.9334
Epoch 25/500
867/867 [==============================] - 0s 377us/step - loss: 127.3903 - val_loss: 392.8654
Epoch 26/500
867/867 [==============================] - 0s 366us/step - loss: 126.6243 - val_loss: 389.8477
Epoch 27/500
867/867 [==============================] - 0s 375us/step - loss: 124.7535 - val_loss: 386.8314
Epoch 28/500
867/867 [==============================] - 0s 373us/step - loss: 121.8172 - val_loss: 383.8070
Epoch 29/500
867/867 [======

Epoch 40/500
867/867 [==============================] - 0s 401us/step - loss: 104.6417 - val_loss: 348.8206
Epoch 41/500
867/867 [==============================] - 0s 369us/step - loss: 101.8131 - val_loss: 345.9692
Epoch 42/500
867/867 [==============================] - 0s 364us/step - loss: 100.4241 - val_loss: 343.1521
Epoch 43/500
867/867 [==============================] - 0s 412us/step - loss: 98.9548 - val_loss: 340.3554
Epoch 44/500
867/867 [==============================] - 0s 414us/step - loss: 98.2319 - val_loss: 337.5939
Epoch 45/500
867/867 [==============================] - 0s 435us/step - loss: 94.6234 - val_loss: 334.8289
Epoch 46/500
867/867 [==============================] - 0s 399us/step - loss: 94.8118 - val_loss: 332.0720
Epoch 47/500
867/867 [==============================] - 0s 375us/step - loss: 93.4585 - val_loss: 329.3548
Epoch 48/500
867/867 [==============================] - 0s 394us/step - loss: 91.3733 - val_loss: 326.6472
Epoch 49/500
867/867 [============

867/867 [==============================] - 0s 397us/step - loss: 76.9343 - val_loss: 297.7537
Epoch 60/500
867/867 [==============================] - 0s 403us/step - loss: 76.1015 - val_loss: 295.2237
Epoch 61/500
867/867 [==============================] - 0s 400us/step - loss: 73.5933 - val_loss: 292.6883
Epoch 62/500
867/867 [==============================] - 0s 390us/step - loss: 74.8109 - val_loss: 290.1845
Epoch 63/500
867/867 [==============================] - 0s 385us/step - loss: 72.9379 - val_loss: 287.6903
Epoch 64/500
867/867 [==============================] - 0s 389us/step - loss: 70.3851 - val_loss: 285.1969
Epoch 65/500
867/867 [==============================] - 0s 386us/step - loss: 69.5442 - val_loss: 282.7177
Epoch 66/500
867/867 [==============================] - 0s 381us/step - loss: 67.2345 - val_loss: 280.2774
Epoch 67/500
867/867 [==============================] - 0s 373us/step - loss: 68.3508 - val_loss: 277.8510
Epoch 68/500
867/867 [============================

867/867 [==============================] - 0s 387us/step - loss: 55.7676 - val_loss: 249.7950
Epoch 80/500
867/867 [==============================] - 0s 389us/step - loss: 55.6422 - val_loss: 247.5384
Epoch 81/500
867/867 [==============================] - 0s 402us/step - loss: 54.2352 - val_loss: 245.2952
Epoch 82/500
867/867 [==============================] - 0s 386us/step - loss: 52.6291 - val_loss: 243.0999
Epoch 83/500
867/867 [==============================] - 0s 382us/step - loss: 53.7355 - val_loss: 240.9407
Epoch 84/500
867/867 [==============================] - 0s 405us/step - loss: 50.8425 - val_loss: 238.7383
Epoch 85/500
867/867 [==============================] - 0s 382us/step - loss: 50.6192 - val_loss: 236.5457
Epoch 86/500
867/867 [==============================] - 0s 394us/step - loss: 47.6598 - val_loss: 234.4073
Epoch 87/500
867/867 [==============================] - 0s 387us/step - loss: 47.6710 - val_loss: 232.2841
Epoch 88/500
867/867 [============================

...

Epoch 415/500
867/867 [==============================] - 0s 415us/step - loss: 23.9411 - val_loss: 134.5974
Epoch 416/500
867/867 [==============================] - 0s 412us/step - loss: 25.0873 - val_loss: 134.3667
Epoch 417/500
867/867 [==============================] - 0s 428us/step - loss: 24.5916 - val_loss: 134.5115
Epoch 418/500
867/867 [==============================] - 0s 412us/step - loss: 23.2164 - val_loss: 134.3033
Epoch 419/500
867/867 [==============================] - 0s 431us/step - loss: 24.5943 - val_loss: 134.4427
Epoch 420/500
867/867 [==============================] - 0s 419us/step - loss: 23.2222 - val_loss: 134.5641
Epoch 421/500
867/867 [==============================] - 0s 400us/step - loss: 25.4906 - val_loss: 134.5298
Epoch 422/500
867/867 [==============================] - 0s 413us/step - loss: 23.9807 - val_loss: 134.6547
Epoch 423/500
867/867 [==============================] - 0s 394us/step - loss: 22.7741 - val_loss: 134.3733
Epoch 424/500
867/867 [=====

867/867 [==============================] - 0s 405us/step - loss: 23.3939 - val_loss: 134.8394
Epoch 435/500
867/867 [==============================] - 0s 445us/step - loss: 24.7092 - val_loss: 134.6692
Epoch 436/500
867/867 [==============================] - 0s 398us/step - loss: 22.4550 - val_loss: 134.4792
Epoch 437/500
867/867 [==============================] - 0s 395us/step - loss: 25.1091 - val_loss: 134.8055
Epoch 438/500
867/867 [==============================] - 0s 423us/step - loss: 24.4466 - val_loss: 134.8014
Epoch 439/500
867/867 [==============================] - 0s 400us/step - loss: 24.4183 - val_loss: 134.6596
Epoch 440/500
867/867 [==============================] - 0s 395us/step - loss: 23.6497 - val_loss: 134.3573
Epoch 441/500
867/867 [==============================] - 0s 417us/step - loss: 23.2108 - val_loss: 134.1786
Epoch 442/500
867/867 [==============================] - 0s 411us/step - loss: 24.1276 - val_loss: 134.2423
Epoch 443/500
867/867 [===================

867/867 [==============================] - 0s 436us/step - loss: 23.9504 - val_loss: 133.8151
Epoch 453/500
867/867 [==============================] - 0s 450us/step - loss: 23.3758 - val_loss: 133.9904
Epoch 454/500
867/867 [==============================] - 0s 433us/step - loss: 25.3898 - val_loss: 134.2102
Epoch 455/500
867/867 [==============================] - 0s 448us/step - loss: 23.1962 - val_loss: 134.1353
Epoch 456/500
867/867 [==============================] - 0s 457us/step - loss: 24.8881 - val_loss: 134.0598
Epoch 457/500
867/867 [==============================] - 0s 450us/step - loss: 23.0569 - val_loss: 133.9234
Epoch 458/500
867/867 [==============================] - 0s 468us/step - loss: 22.8805 - val_loss: 134.0086
Epoch 459/500
867/867 [==============================] - 0s 441us/step - loss: 24.2168 - val_loss: 134.0137
Epoch 460/500
867/867 [==============================] - 0s 427us/step - loss: 23.2311 - val_loss: 133.8675
Epoch 461/500
867/867 [===================

867/867 [==============================] - 0s 404us/step - loss: 23.7025 - val_loss: 133.7718
Epoch 470/500
867/867 [==============================] - 0s 380us/step - loss: 23.4820 - val_loss: 133.9017
Epoch 471/500
867/867 [==============================] - 0s 368us/step - loss: 23.7848 - val_loss: 133.7475
Epoch 472/500
867/867 [==============================] - 0s 392us/step - loss: 25.0736 - val_loss: 134.0695
Epoch 473/500
867/867 [==============================] - 0s 375us/step - loss: 23.1283 - val_loss: 134.0026
Epoch 474/500
867/867 [==============================] - 0s 411us/step - loss: 23.9572 - val_loss: 134.3195
Epoch 475/500
867/867 [==============================] - 0s 389us/step - loss: 23.8290 - val_loss: 134.3770
Epoch 476/500
867/867 [==============================] - 0s 380us/step - loss: 24.6860 - val_loss: 134.3845
Epoch 477/500
867/867 [==============================] - 0s 399us/step - loss: 23.5755 - val_loss: 134.1903
Epoch 478/500
867/867 [===================

867/867 [==============================] - 0s 398us/step - loss: 24.2543 - val_loss: 133.9255
Epoch 489/500
867/867 [==============================] - 0s 397us/step - loss: 23.3307 - val_loss: 133.8842
Epoch 490/500
867/867 [==============================] - 0s 395us/step - loss: 25.3261 - val_loss: 134.0910
Epoch 491/500
867/867 [==============================] - 0s 432us/step - loss: 24.5108 - val_loss: 134.2586
Epoch 492/500
867/867 [==============================] - 0s 420us/step - loss: 25.0003 - val_loss: 134.1386
Epoch 493/500
867/867 [==============================] - 0s 373us/step - loss: 23.2733 - val_loss: 134.0241
Epoch 494/500
867/867 [==============================] - 0s 381us/step - loss: 22.3744 - val_loss: 133.8934
Epoch 495/500
867/867 [==============================] - 0s 395us/step - loss: 23.4154 - val_loss: 133.8456
Epoch 496/500
867/867 [==============================] - 0s 369us/step - loss: 23.3549 - val_loss: 133.8369
Epoch 497/500
867/867 [===================

In [0]:
# model predictions on test data
pred=model.predict(testX)
pred=pred[:,0]

# backtesting
vix=data.sort_index(ascending=False)['Close'][0:test_size]    # benchmark Close price
# convert prediction into a series
pred=pd.Series(pred,index=vix.index)                                 

# plot vix price and prediction
configure_plotly_browser_state()
vix=Scatter(x=vix.index,y=vix,name='VIX')
prediction=Scatter(x=pred.index,y=pred,name='prediction')
iplot([vix,prediction])